In [17]:
%matplotlib inline
import matplotlib as mlp
import scipy
import numpy as np
import sympy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import scale
from scipy.stats.stats import spearmanr 
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
from sklearn import metrics
import scipy.sparse as sps
import math
from bs4 import BeautifulSoup
import requests
import re
import os
import fileinput
# import sys



In [18]:
import inspect

def __LINE__():
    try:
        raise Exception
    except:
        return sys.exc_info()[2].tb_frame.f_back.f_lineno

def __FILE__():
    return inspect.currentframe().f_code.co_filename


In [20]:


def clean_text(text):
    """ Removes white-spaces before, after, and between characters

    :param text: the string to remove clean
    :return: a "cleaned" string with no more than one white space between
    characters
    """
    return ' '.join(text.split())

df_book = pd.DataFrame(data={'Book Name' : [],'Author' : [], 'Awards List' : [], 'Genres' : [], 'Characters' : []})

# reload(sys)
# sys.setdefaultencoding('utf-8')

#TODO :
# 1. Clean awards to trim ..more , ..less
# 2. Delimit by commas 
# 3. Make a pandas dataframe and put into it the data for the book

    
# with open('book_was_better.txt','r') as book_better:
#     for url_first in book_better:
i = 0
fin = fileinput.input(files=('books_becoming_movies_2016.txt'),mode='r')
for url_first in fin:

# if 1:
    url_first  = url_first.split('\t')[0]
    try:
#        if (i>700):
            book_page = requests.get(url_first)

            # Strain the items from the book page
            soup = BeautifulSoup(book_page.content,"lxml")
            print i
            #Book Name

            book_name = soup.find_all('h1',attrs={"class":'bookTitle'})
            book_name = book_name[0].text
            book_name = clean_text(book_name)
            print book_name

            #Author
            author = soup.find_all('a',attrs={"class":'authorName'})[0].text

            #Awards
            awards_string = ""
            for row in soup.find_all('div',attrs={"class":'infoBoxRowItem',"itemprop":'awards'}):
                    awards_string = row.text

            #Genres
            genre_list = []
            for row in soup.find_all('a',attrs={"class":'actionLinkLite bookPageGenreLink'}):
                    genre_list.append(row.text)

            #Characters
            character_list = []
            for row in soup.find_all('a', {'href': re.compile(r'/characters')}):
                character_list.append(row.text)

            #Other editions
            m = ""
            other_editions = soup.find_all('a', {'href': re.compile(r'/work/editions/')})[1].text
            if other_editions:
                m = re.search('.*\((.*)\)', other_editions)
            if m:
                other_editions = int(m.group(1))

            #Getting number of votes for [5 4 3 2 1] star ratings
            script_text = []
            m = ""
            for row in soup.find_all('script',attrs={'type':'text/javascript+protovis'}):
                script_text.append(str(row))
            if len(script_text) >0:
                m = re.search('.*renderRatingGraph\(\[(.*)\]\).*', script_text[0])
            if m:
                rating_values_str = m.group(1)
            rating_values_list = [ int(x) for x in rating_values_str.split(",") ]

            #Average star rating
            star = soup.find_all('span',attrs={"class":'average', "itemprop":"ratingValue"})[0].text
            print star

            #Pages
            pages = soup.find_all('span',attrs={"itemprop":"numberOfPages"})
            if len(pages) > 0:
                pages = pages[0].text
            else:
                pages = ""

            #publish_date
            publish_date = soup.find_all('nobr',attrs={"class":'greyText'})
            if len(publish_date) > 0:
                publish_date = publish_date[0].text
                m = re.search('\(.*d\s(.*)\)', publish_date)
                if m:
                    publish_date = m.group(1)
            else: 
                publish_date = soup.find_all('nobr',attrs={"class":'greyText'})
                if len(publish_date) > 0:
                    publish_date = publish_date[0].text
                else:
                    publish_date = ""
                
            print publish_date

            #num_reviews
            num_reviews = soup.find_all('span',attrs={"class":"count"})[0].text
            print num_reviews
            m = re.search('(.*?)\s', num_reviews)
            if m:
                num_reviews = int(m.group(1).replace(',', ''))

            print ("\nnum_reviews = ",num_reviews)

            df = pd.DataFrame({'Book Name':book_name, 'Author':author, 'Awards List':awards_string,
                               'Genres':[genre_list], 'Characters':[character_list], 'Other editions':other_editions,
                              'Star votes':[rating_values_list], 'Stars':star, 'Num_reviews':num_reviews,
                              'publish_date':publish_date, 'Pages':pages})
            df_book = df_book.append(df,ignore_index=True)

            if (i%20 == 0):
                df_book.index += (i-19)
                df_book.to_csv('goodreads_data_test.csv', mode='a', header=(True if i<21 else False), encoding="utf-8")
                df_book = pd.DataFrame()
            i +=1

    except Exception as exception:
#                 df.to_csv('skipped.csv', mode='a', header=False, ignore_index=True, encoding="utf-8")
            f = open('goodreads_skipped', 'a')
            print type(exception).__name__
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            f.write(url_first + " "+type(exception).__name__+" "+str(exc_tb.tb_lineno)+"\n")
            f.close()
            i +=1


df_book.index += (i-19)
df_book.to_csv('goodreads_data_test.csv', mode='a', header=(True if i<21 else False), encoding="utf-8")
df_book = pd.DataFrame()

fin.close()
df_book

0
Miss Peregrine’s Home for Peculiar Children (Miss Peregrine’s Peculiar Children #1)
3.86

48,613
('\nnum_reviews = ', u'48,613')
1
Me Before You (Me Before You #1)
4.28
January 5th 2012
66,541
('\nnum_reviews = ', u'66,541')
2
Fantastic Beasts and Where to Find Them (Hogwarts Library)
3.94
2001
5,176
('\nnum_reviews = ', u'5,176')
3
Allegiant (Divergent #3)
3.64

61,344
('\nnum_reviews = ', u'61,344')
4
The Girl on the Train
3.87

82,435
('\nnum_reviews = ', u'82,435')
5
The 5th Wave (The 5th Wave #1)
4.13

25,249
('\nnum_reviews = ', u'25,249')
6
Throne of Glass (Throne of Glass #1)
4.24
2012
25,061
('\nnum_reviews = ', u'25,061')
7
Shatter Me (Shatter Me #1)
4.02

19,256
('\nnum_reviews = ', u'19,256')
8
The Light Between Oceans
3.99
2012
26,745
('\nnum_reviews = ', u'26,745')
9
Ready Player One
4.31

44,630
('\nnum_reviews = ', u'44,630')
10
The BFG
4.21
1982
7,100
('\nnum_reviews = ', u'7,100')
11
The Darkest Minds (The Darkest Minds #1)
4.27

10,482
('\nnum_reviews = ', u'10,482

""
